In [1]:
cd ..

d:\Project\ThesisProd


In [2]:
import os
import time
import numpy as np

from dotenv import load_dotenv
from typing import Any, List, Dict, Callable, Union, Optional, Tuple, Literal

from core.llm.AssistantLLM import AssistantBot
from core.ingestion.preprocessing.hyde.HyDe import HyDETransformer
from core.ingestion.preprocessing.multistep.MultiStep import MultiStepTransformer

Custom LLM initialized with model: gpt-3.5-turbo
Custom LLM initialized with model: gpt-3.5-turbo


In [3]:
transformer = MultiStepTransformer()
response = transformer.transform(text=[["Hello, how are you?", "What is the purpose of life?"]])
print(response)

[["2. What is the main question being asked in 'Hello, how are you?'?", "3. How can one respond to the question 'Hello, how are you?'?"], ['2. How do individuals define their own purpose in life?', '3. How can one find meaning and fulfillment in their life?']]
